Run HTML in a notebook.

In [ ]:
from IPython.core.display import HTML

HTML("""
<div>
    <video width=400 controls="controls">
        <source src="assets/video/ex1.mp4" type="video/mp4">
    </video>
    <video width=400 controls="controls">
        <source src="assets/video/ex2.mp4" type="video/mp4">
    </video>
</div>
""")

Create a VideoCapture object from an Ocean Exploration Challenge video clip and operate (level adjust and blur) on each frame of the video.
The VideoCapture object will have 9000 frames (5m x 60sec/m x 30fps), or 2,073,600,000 RGB pixels (640 x 360 x 9000). That's a lot of operations.

Operate on the original color video clip.

In [ ]:
import cv2
import numpy as np

# create a VideoCapture object
cap = cv2.VideoCapture('assets/video/ex1.mp4')

i = 0

# loop until the end of the video
while (cap.isOpened()):

    # capture the video frame-by-frame
    ret, frame = cap.read()
    
    # adjust the image levels
    frame_adj = cv2.convertScaleAbs(frame, alpha=1.95, beta=0)   # 1-3, 0-100
    
    # blur the image
    frame_blur = cv2.GaussianBlur(frame_adj, (9,9), 0)
    
    # DRAW A RECTANGLE
    cv2.rectangle(frame_blur,(100+i,50+i),(300+i,150+i),color=(0,0,255),thickness=2)
    i = i + 1
    cv2.putText(frame_blur, text="Shrimp Video", org=(20,40), fontFace=3, fontScale=1, color=(255,255,255), thickness=2)
    
    # display the frame
    #cv2.imshow('my video', frame_adj)
    cv2.imshow('my video', frame_blur)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()  

Operate on a grayscale conversion of the original color video clip.

In [ ]:
import cv2
import numpy as np

# create a VideoCapture object
cap = cv2.VideoCapture('assets/video/ex1.mp4')

# loop until the end of the video
while (cap.isOpened()):

    # capture the video frame-by-frame
    ret, frame = cap.read()
    
    # convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # adjust the image levels
    gray_adj = cv2.convertScaleAbs(gray, alpha=1.95, beta=0)
    
    # blur the image
    gray_blur = cv2.GaussianBlur(gray_adj, (9,9), 0)
    
    # display the frame
    #cv2.imshow('my video', gray_adj)
    cv2.imshow('my video', gray_blur)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

It is critical to understand two important concepts when learning computer vision and image processing techniques.
* thresholding = segmenting foreground from background (which pixels belong to the foreground and which belong to the backbround)
* edge detecting = 

Color separation for segmentation.

In [ ]:
import cv2
import numpy as np

# create a VideoCapture object
cap = cv2.VideoCapture('assets/video/ex1.mp4')

# loop until the end of the video
while (cap.isOpened()):

    # capture the video frame-by-frame
    ret, frame = cap.read()
    
    # split the color channels
    (B, G, R) = cv2.split(frame)
    #cv2.imshow("Red", R)
    #cv2.imshow("Green", G)
    #cv2.imshow("Blue", B)
    
    # visualize each channel in color
    zeros = np.zeros(frame.shape[:2], dtype="uint8")
    frame_R = cv2.merge([zeros, zeros, R])
    frame_G = cv2.merge([zeros, G, zeros])
    frame_B = cv2.merge([B, zeros, zeros])
    cv2.imshow("Red", frame_R)
    #cv2.imshow("Green", frame_G)
    #cv2.imshow("Blue", frame_B)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(frame.shape[:2])
print(frame.shape)
print(zeros)

Pre-process the image:
* Get the red data from the frame.
* Convert the frame to grayscale.
* Apply a 7×7 Gaussian blur. Applying Gaussian blurring helps remove some of the high frequency edges in the image that we are not concerned with and allow us to obtain a more “clean” segmentation.
* Apply the threshold.

In [ ]:
import cv2
import numpy as np

# create a VideoCapture object
cap = cv2.VideoCapture('assets/video/ex1.mp4')

# loop until the end of the video
while (cap.isOpened()):
    cap.set(1, 500)

    # capture the video frame-by-frame
    ret, frame = cap.read()
    
    # adjust the image levels
    frame_adj = cv2.convertScaleAbs(frame, alpha=1.95, beta=0)   # 1-3, 0-100
    
    # split the color channels
    (B, G, R) = cv2.split(frame)
    #cv2.imshow("Red", R)
    #cv2.imshow("Green", G)
    #cv2.imshow("Blue", B)
    
    # adjust the image levels
    R_adj = cv2.convertScaleAbs(R, alpha=2.00, beta=0)
    
    # blur the image
    R_blur = cv2.GaussianBlur(R_adj, (7,7), 0)
    
    # simple threshold
    #(T, R_thresh) = cv2.threshold(R_blur, 200, 255, cv2.THRESH_BINARY_INV)
    (T, R_thresh) = cv2.threshold(R_blur, 200, 255, cv2.THRESH_BINARY)
    
    # view only the masked regions in the image
    R_masked = cv2.bitwise_and(R_adj, R_adj, mask=R_thresh)
    frame_masked = cv2.bitwise_and(frame_adj, frame_adj, mask=R_thresh)
    
    # using cv2.Canny() for edge detection.
    edge_detect = cv2.Canny(R_masked, 100, 200)
    
    # display the frame
    #cv2.imshow('my video', R)
    #cv2.imshow('my video', R_qadj)
    #cv2.imshow('my video', R_blur)
    cv2.imshow('my video', R_thresh)
    #cv2.imshow('my video', R_masked)
    #cv2.imshow('my video', frame_masked)
    #cv2.imshow('my video', edge_detect)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('assets/output.mp4', -1, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,0)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

#read image
img_raw = cv2.imread('assets/0922-1.tiff')

#select ROI function
roi = cv2.selectROI(img_raw)

#print rectangle points of selected roi
print(roi)

#Crop selected roi from raw image
roi_cropped = img_raw[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]

#show cropped image
cv2.imshow("ROI", roi_cropped)

#cv2.imwrite("crop.jpeg",roi_cropped)

#hold window
cv2.waitKey(0)
cv2.destroyAllWindows()

(133, 42, 200, 126)
